<a href="https://colab.research.google.com/github/chris-bhaila/ANAIS-2025/blob/main/Day_1_GNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authors: A. Lupidi (alisia.lupidi@cs.ox.ac.uk), T. Reu (teodora.reu@cs.ox.ac.uk)

# ANAIS Geometric Deep Learning
## Practical 1: GNNs

*Welcome to our first practical* 🚀 \
This notebook will focus on Graph Neural Networks.
In the following sections, we will:
- Part 0: a quick refresh on theory and set up
- Part 1: building a GNN
- Part 2: exploring expressivity of GNNs
- Part 3: exploring equivarance and invariance

This content is adapted from the Geometric Deep Learning course at the University of Cambridge (credits P. Lio, P. Veličković, C. K. Joshi, C. Harris, R. Torné, C. Bodnar) and the Graph Representation Learning course at the University of Oxford (credits İ. Ceylan, F. Barbero, X. Huang).

References:
- The Geometric Deep Learning Book (M. M. Bronstein, J. Bruna, T. Cohen, P. Veličković 2021)

# **Your tasks**
The aim of this tutorial is to provide you with a practical understanding of the theory and coding required in Parts 1-3.
Practically, you will need to complete the GIN Encoder and Decoder classes in `Message Passing GNN (GIN) + MLP decoder` and answer the question in section `1. Exercise`.
Throughout the tutorial, we have dispensed nuggets of theory, demostrated in code. Try tackling the coding exercise on your own or in groups, don't worry if you don't finish it. Also, do spend some time and thinking on the definitions and readings, we will have an interactive discussion at the end. Teo and Alisia will demonstrate will demonstrate the solution to the exercise at the end of the turorial, don't hesitate to ask any questions!

We hope you enjoy the exercise and the reading.
Happy GNNs🚀




# 📖 Part 0.a: a refresh on theory

<img src="https://github.com/metalisia/gdl_images/blob/main/gldp1f1.png?raw=1">

#### What is a Graph?

A graph is a tuple $\mathbb{G}=(\mathbb{V}, \mathbb{E})$  where $\mathbb{V}$ is a set of nodes (or vertices) and $\mathbb{E} ⊆ (\mathbb{V} × \mathbb{V})$ is a 2-tuple set of the edges in the graph. The element $(\mathbb{u}, \mathbb{v}) ∈ \mathbb{E}$ is a directed from $\mathbb{u}$ to $\mathbb{v}$ and indicates a relationship between nodes $\mathbb{u}$ and $\mathbb{v}$.

The most convenient way to represent a graph is by adjeciency matrix. Given a graph $\mathbb{G}$ defined as above where $|\mathbb{V}| = n$ nodes, $A \in \mathbb{R}^{n \times n}$ is a symmetric adjacency matrix where is $a_{i,j}$ is the weight of the edge between nodes $v_i$ and $v_j$. If $(v_i, v_j) \notin E$ then $a_{i,j} = 0$.

Graphs can store layers of information in the nodes and edges (features). For graphs with node features, each node $v_i \in \mathbb{V}$ has an associated $d$-dimensional feature vector $\mathbf{x_i} \in \mathbb{R}^{d}$. Then the feature matrix $\mathbf{X} \in \mathbb{R}^{n \times d}$ can be used to represent the feature vectors for every node in the graph.


#### What is a Graph Neural Network?
A GNN is a Neural Network architecture built for handling structured data in the shape of graphs. GNNs update node representations by aggregating information from neighboring nodes and edges (message passing). After several rounds of aggregation, each node’s representation encodes information about its local graph structure. This makes them powerful tools for tasks like node / edge / graph-level prediction or classification.

# ⚙️ Part 0.b: set up

The code below is setting up the environment for this practical along with defining some handy functions for reproducibility,

**❗️Note:** You will need a GPU to complete this practical. Remember to click `Runtime -> Change runtime type`, and set the `hardware accelerator` to **GPU**.

In [ ]:
!pip install torch==2.2.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html
!pip install networkx

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in links: https://download.pytorch.org/whl/torch_stable.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.2 MB/s e

In [ ]:
!pip install -U pip wheel setuptools


In [ ]:
!python -m pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.5.1+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 65.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 65.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.8/994.8 kB 28.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [torch-cluster]


In [ ]:
!python -m pip install --index-url https://download.pytorch.org/whl/cu121 torch==2.5.1+cu121
!python -m pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!python -m pip install torch-geometric

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 36.2 MB/s  0:00:11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 120.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 126.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 16.9 MB/s  0:00:19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 30.1 MB/s  0:00:09
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 66.4 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 51.0 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 56.8 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 63.9 MB/s  0:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 57.8 MB/s  0:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 MB 70.2

In [ ]:
import os
import time
import random
import numpy as np

from scipy.stats import ortho_group

import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, BatchNorm1d, Module, Sequential

import torch_geometric
from torch_geometric.data import Data
from torch_geometric.data import Batch
from torch_geometric.datasets import QM9
import torch_geometric.transforms as T
from torch_geometric.utils import remove_self_loops, to_dense_adj, dense_to_sparse
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_scatter import scatter

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from google.colab import files
from IPython.display import HTML

print("PyTorch version {}".format(torch.__version__))
print("PyG version {}".format(torch_geometric.__version__))

PyTorch version 2.5.1+cu121
PyG version 2.7.0


In [ ]:
def seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed(0)

Great! We are ready to dive into the practical!



#🥇 Part 1: How to build a (Message Passing) Graph Neural Network


<!-- ![](https://drive.google.com/uc?id=1Wdgdq606XW1MelvcU1nW5CxWe1rHsWt1) -->
<img src="https://github.com/chaitjo/dump/raw/main/gnn-layers.png">

## Message Passing Neural Network for Link Prediction on Cora

**Goal.** Given the Cora citation graph $\mathbb{G}=(\mathbb{V},\mathbb{E})$, learn a model that predicts whether a link exists between a pair of nodes $(\mathbb{u},\mathbb{v})$. This is a **binary classification** task over candidate node pairs.

Cora provides node features $x_i \in \mathbb{R}^{d_n}$ for each node $i\in V$. It typically has **no explicit edge features**; therefore we either:
- omit edge features from the message function, or
- define constant edge features $e_{ij}=\mathbf{0}\in\mathbb{R}^{d_e}$ (optional).

---

### Encoder: $L$-Layer Message Passing Network

We compute hidden node representations $h_i^\ell \in \mathbb{R}^{d}$ for $\ell=0,\dots,L$.

#### Input Projection
Project input node features to the hidden dimension:
$$
h_i^{0} = W_{\text{in}} x_i,
\qquad W_{\text{in}} \in \mathbb{R}^{d \times d_n}.
$$

#### MPNN Layer
For each layer $\ell \to \ell+1$, update node embeddings via message passing:
$$
h_i^{\ell+1} = \phi\Bigg(
h_i^{\ell},
\ \ \oplus_{j\in\mathcal{N}(i)} \psi\left(h_i^{\ell}, h_j^{\ell}, e_{ij}\right)
\Bigg),
$$
where:
- $\mathcal{N}(i)$ is the neighbor set of node $i$,
- $\oplus$ is a permutation-invariant aggregator (we use summation),
- $\psi$ is a message MLP,
- $\phi$ is an update MLP.

Pedagogical decomposition:

1) **Message**
$$
m_{ij}^{\ell}=\psi\left(h_i^\ell, h_j^\ell, e_{ij}\right),
\qquad \psi:\mathbb{R}^{2d+d_e}\to\mathbb{R}^{d}.
$$

2) **Aggregate**
$$
m_i^\ell = \sum_{j \in \mathcal{N}(i)} m_{ij}^{\ell}.
$$

3) **Update**
$$
h_i^{\ell+1} = \phi\left(\left[h_i^\ell \ \| \ m_i^\ell\right]\right),
\qquad \phi:\mathbb{R}^{2d}\to\mathbb{R}^{d},
$$
where $\cdot \| \cdot$ denotes concatenation.

After $L$ layers, define the final node embedding:
$$
z_i := h_i^{L}\in\mathbb{R}^{d}.
$$

**Note.** For link prediction we do **not** apply global pooling; we require node-level embeddings $z_i$ to score specific pairs $(u,v)$.

---

### Decoder: Edge Scoring / Prediction Head

For a candidate node pair $(u,v)$, compute a scalar **logit** $s_{uv}\in\mathbb{R}$ and probability $\hat p_{uv}\in(0,1)$.

A common MLP-based decoder uses pairwise features:
$$
s_{uv} = g\left(\left[z_u \ \| \ z_v \ \| \ (z_u\odot z_v)\right]\right),
\qquad g:\mathbb{R}^{3d}\to\mathbb{R},
$$

$$
\hat p_{uv} = \sigma(s_{uv}),
$$
where $\odot$ is elementwise product and $\sigma(\cdot)$ is the sigmoid.

(Alternative decoders include dot-product $s_{uv}=z_u^\top z_v$ or bilinear scoring.)

---

### Link Prediction Splits (Train / Val / Test)

We create disjoint sets of **positive** edges:
$$
E^+_{\text{train}},\ E^+_{\text{val}},\ E^+_{\text{test}}
\quad \text{with} \quad
E^+_{\text{train}} \cup E^+_{\text{val}} \cup E^+_{\text{test}} \subseteq E.
$$

During training, message passing is performed on a **training graph**
$$
G_{\text{mp}} = (V, E_{\text{mp}}),
$$
where $E_{\text{mp}}$ excludes held-out validation/test positives to prevent leakage.

For supervised learning and evaluation, we also require **negative** edges:
$$
E^-_{\text{train}} \subset (V\times V)\setminus E,
\quad
E^-_{\text{val}} \subset (V\times V)\setminus E,
\quad
E^-_{\text{test}} \subset (V\times V)\setminus E,
$$
often created via random negative sampling.


---

### Loss Function: Binary Cross-Entropy with Negative Sampling

At each epoch, sample negatives $E^-_{\text{train}}$ (commonly $|E^-_{\text{train}}| = |E^+_{\text{train}}|$). Define labels:
$$
y_{uv}=
\begin{cases}
1 & (u,v)\in E^+_{\text{train}}\\
0 & (u,v)\in E^-_{\text{train}}
\end{cases}
$$
and logits $s_{uv}$ from the decoder.

The binary cross-entropy loss is:
$$
\mathcal{L}_{\text{BCE}}=
-\sum_{(u,v)\in E^+_{\text{train}}\cup E^-_{\text{train}}}
\left[
y_{uv}\log \sigma(s_{uv}) + (1-y_{uv})\log (1-\sigma(s_{uv}))
\right].
$$

This objective trains the encoder (message passing) and decoder end-to-end.

---

### Evaluation Metrics

We evaluate on $\big(E^+_{\text{val}}\cup E^-_{\text{val}}\big)$ and $\big(E^+_{\text{test}}\cup E^-_{\text{test}}\big)$ using ranking/classification metrics such as:
- **ROC-AUC**
- **Average Precision (AP)**


---
### The dataset
#### Cora
The dataset we will be using is Cora (https://graphsandnetworks.com/the-cora-dataset/). The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.
In graph terms, each paper is a node and the citation-relationship determines directed edges between papers.

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

transform = T.Compose([
    T.NormalizeFeatures(),
    T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.10,
        is_undirected=True,
        add_negative_train_samples=False,  # we will negative-sample each epoch
    ),
])

dataset = Planetoid(root="data/Planetoid", name="Cora", transform=transform)
train_data, val_data, test_data = dataset[0]

train_data = train_data.to(device)
val_data   = val_data.to(device)
test_data  = test_data.to(device)

print(train_data)


Device: cpu


Processing...
Done!


Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[4488], edge_label_index=[2, 4488])


# Exercise

What are the $\psi$ and $\phi$ functions for GIN? How do the message, aggregate, and update functions look like?



#### Message Passing GNN (GIN) + MLP decoder

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GINConv
from torch_geometric.utils import negative_sampling

from sklearn.metrics import roc_auc_score, average_precision_score

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

class GINEncoder(nn.Module):
    def __init__(self, in_channels: int, hidden: int = 128, out_channels: int = 128, dropout: float = 0.2):
        super().__init__()

        self.dropout = dropout

       mlp1 =


class EdgeDecoder(nn.Module):
    """MLP over pairwise node embeddings -> edge logit."""
    def __init__(self, z_dim: int, hidden: int = 128, dropout: float = 0.2):
        """ Insert your code here! """
        continue

    def forward(self, z, edge_label_index):
      """ Insert your code here! """
        pass


class LinkPredictor(nn.Module):
    def __init__(self, in_channels: int, hidden: int = 128, z_dim: int = 128, dropout: float = 0.2):
        super().__init__()
        self.encoder = GINEncoder(in_channels, hidden, z_dim, dropout)
        self.decoder = EdgeDecoder(z_dim, hidden, dropout)

    def encode(self, x, edge_index):
        return self.encoder(x, edge_index)

    def decode(self, z, edge_label_index):
        return self.decoder(z, edge_label_index)


model = LinkPredictor(in_channels=train_data.num_features, hidden=128, z_dim=128, dropout=0.2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()


#### Train / Eval

In [ ]:
@torch.no_grad()
def evaluate(split_data):
    model.eval()
    z = model.encode(split_data.x, split_data.edge_index)
    logits = model.decode(z, split_data.edge_label_index)
    probs = torch.sigmoid(logits).cpu().numpy()
    y = split_data.edge_label.cpu().numpy()

    auc = roc_auc_score(y, probs)
    ap = average_precision_score(y, probs)
    return auc, ap


def train_one_epoch():
    model.train()
    optimizer.zero_grad()

    z = model.encode(train_data.x, train_data.edge_index)

    # Positive supervision edges provided by RandomLinkSplit
    pos_edge_index = train_data.edge_label_index
    num_pos = pos_edge_index.size(1)

    # Fresh negative edges each epoch
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index,
        num_nodes=train_data.num_nodes,
        num_neg_samples=num_pos,
        method='sparse',
    )

    edge_label_index = torch.cat([pos_edge_index, neg_edge_index], dim=1)
    edge_label = torch.cat([
        torch.ones(num_pos, device=device),
        torch.zeros(neg_edge_index.size(1), device=device),
    ], dim=0)

    logits = model.decode(z, edge_label_index)
    loss = criterion(logits, edge_label)
    loss.backward()
    optimizer.step()

    return float(loss)


best_val_auc = -1.0
best_state = None

for epoch in range(1, 501):
    loss = train_one_epoch()

    if epoch % 10 == 0:
        val_auc, val_ap = evaluate(val_data)
        test_auc, test_ap = evaluate(test_data)
        print(f"Epoch {epoch:03d} | Loss {loss:.4f} | Val AUC {val_auc:.4f} AP {val_ap:.4f} | Test AUC {test_auc:.4f} AP {test_ap:.4f}")

        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

if best_state is not None:
    model.load_state_dict(best_state)

val_auc, val_ap = evaluate(val_data)
test_auc, test_ap = evaluate(test_data)
print(f"\nBest Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}")
print(f"Final Test AUC: {test_auc:.4f} | Test AP: {test_ap:.4f}")


**Note**
Golden scores: These scores help you assess if you've coded up the GIN correctly. Try to match them -- or beat them!

`Best Val AUC: 0.8501 | Val AP: 0.8558
Final Test AUC: 0.8533 | Test AP: 0.8483`

#🥈 Part 2: Expressivity of GNNs

## Weisfeiler–Lehman (1-WL Hashing)

We will explore the expressivity of GNNs with implementing the **1-dimensional Weisfeiler–Lehman (1-WL)** algorithm (also called **color refinement**) and use it to study the **applicability and limitations** of message passing GNNs for distinguishing graphs.



### Why does WL matters for GNN expressivity?

Most common GNN architectures (GCN, GraphSAGE, GAT, GIN, and many MPNNs) follow the **message passing** paradigm:

- each node updates its representation using a *permutation-invariant* aggregation of its neighbors’ representations,
- repeated over $L$ layers.

The 1-WL algorithm is a classical procedure that iteratively refines **discrete node labels** based on neighbor labels. The update structure is closely aligned with message passing. This connection motivates a key principle:

- **1-WL is an upper bound on the distinguishing power of standard message-passing GNNs.**
This means that if **1-WL cannot distinguish** two graphs, then a standard message passing GNN (without extra node identifiers/positional encodings) cannot be guaranteed to distinguish them either. If **1-WL does distinguish** two graphs, then a sufficiently expressive message passing GNN (with injective aggregation/update) *can* separate them in principle.


---


## The 1-WL color-refinement algorithm

Let $G=(V,E)$ be a graph. Each node $i \in V$ starts with a discrete label (color) $c_i^{(0)}$.
If graphs have node labels, use those labels as $c_i^{(0)}$. Otherwise, a common default is **all nodes share the same initial label** (or optionally use node degree as initialization).

### Iterative refinement

For iterations $t=0,1,\dots,T-1$, update each node label using its current label and the **multiset** of neighbor labels:
$$
c_i^{(t+1)} \;=\; \mathrm{HASH}\Big( c_i^{(t)}, \text{multiset}\big(\{ c_j^{(t)} : j \in \mathcal{N}(i) \}\big),
$$
where:
- $\mathcal{N}(i)$ is the set of neighbors of node $i$,
- $\text{multiset}(\cdot)$ denotes a multiset (neighbor labels can repeat),
- `HASH` is a deterministic mapping from *signatures* to new discrete labels

To represent the neighbor multiset deterministically, sort it and assign a new integer label to each unique signature. Then, replace node labels by these new integers to obtain $c_i^{(t+1)}$.

The algorithm terminates when the labels stabilize (no changes), or
a fixed number of rounds \(T\) is reached.

In practice, using a small \(T\) is useful when you want to compare against \(L\)-layer GNNs.


If for some \(t\),
$$
C^{(t)}(G)\neq C^{(t)}(H),
$$
then 1-WL distinguishes \(G\) and \(H\), meaning they are definitely **non-isomorphic**.

**Note:** identical WL fingerprints (sorted $C_j$) do **not** guarantee isomorphism, as 1-WL is not complete.




## 1-WL as a test for GNN Expressivity

The **1-dimensional Weisfeiler–Lehman (1-WL) algorithm**, often called **color refinement**, serves as the theoretical "gold standard" for measuring the expressivity of Graph Neural Networks. By implementing 1-WL, we can rigorously study the fundamental limitations of message passing GNNs in distinguishing non-isomorphic graph structures.

### The Link Between WL and GNNs

Most modern GNN architectures (e.g., GCN, GAT, GraphSAGE) are categorized as **Message Passing Neural Networks (MPNNs)**. Their update mechanism mirrors the WL process:
* **Local Aggregation:** Each node updates its state using a *permutation-invariant* function over its neighborhood.
* **Iterative Refinement:** Representations are refined over $L$ successive layers.



**The Key Principle:**
The 1-WL test provides a **theoretical upper bound** on the distinguishing power of standard MPNNs.
* If 1-WL cannot distinguish two graphs $G$ and $H$, no standard MPNN can distinguish them, regardless of its depth or hidden dimension.
* A GNN achieves "WL-equivalent" power only if its aggregation and update functions are **injective** (e.g., the Graph Isomorphism Network, or GIN).


The graphs below are non-isomorphic but have the same colouring.
<img src="https://github.com/metalisia/gdl_images/blob/main/1wl.png?raw=1" width="600">


## The 1-WL Algorithm: Step-by-Step

Consider a graph $G=(V, E)$. The algorithm proceeds by iteratively refining discrete node labels (colors).

### 1. Initialization
Each node $i \in V$ is assigned an initial color $c_i^{(0)}$.
* If the dataset provides node features/labels, use those.
* If the graph is unlabeled, initialize all nodes with the same constant value, $c_i^{(0)} = 1$.

### 2. Iterative Color Refinement
For each iteration $t = 0, 1, \dots, T-1$, we update the color of node $i$ based on its current state and the **multiset** of its neighbors' colors:

$$c_i^{(t+1)} = \text{HASH} \left( c_i^{(t)}, \left\{\!\left\{ c_j^{(t)} \mid j \in \mathcal{N}(i) \right\}\!\right\} \right)$$

Where:
* $\{\!\{ \cdot \}\!\}$ denotes a **multiset**, capturing both the values and the frequencies of neighbor labels.
* $\text{HASH}$ is an injective function that maps each unique (node color, neighbor multiset) pair to a unique new integer.

### 3. Practical Implementation
In code, the "Hashing" step is typically performed by:
1.  Collecting the labels of all neighbors.
2.  Sorting the labels to ensure the representation is **permutation-invariant**.
3.  Concatenating the node’s own label with the sorted neighbor list to create a unique "signature."
4.  Mapping these signatures to new, smaller integers using a hash table.

---

## Graph Distinguishability and Isomorphism

After $T$ iterations, we define the **graph fingerprint** as the sorted multiset of all node colors: $\text{f}(G) = \{\!\{ c_i^{(T)} \mid i \in V \}\!\}$.

* **Non-Isomorphism Check:** If $\text{f}(G) \neq \text{f}(H)$, the graphs are guaranteed to be **non-isomorphic**.
* **The 1-WL Limit:** If $\text{f}(G) = \text{f}(H)$, the graphs *might* be isomorphic. However, 1-WL is "incomplete"; it famously fails to distinguish certain structures like circular skip links or regular graphs with the same degree.

In [ ]:
import random
import numpy as np
import networkx as nx
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GINConv, global_add_pool


def wl1_colors_nx(G, num_iters=5, init="uniform"):
    nodes = list(G.nodes())
    if init == "uniform":
        colors = {i: 0 for i in nodes}
    elif init == "degree":
        colors = {i: G.degree[i] for i in nodes}
    else:
        raise ValueError("init must be 'uniform' or 'degree'")

    colors_by_iter = [colors.copy()]
    for _ in range(num_iters):
        sigs = {}
        for i in nodes:
            neigh_cols = sorted(colors[j] for j in G.neighbors(i))
            sigs[i] = (colors[i], tuple(neigh_cols))

        uniq = sorted(set(sigs.values()))
        mapping = {sig: idx for idx, sig in enumerate(uniq)}
        colors = {i: mapping[sigs[i]] for i in nodes}
        colors_by_iter.append(colors.copy())
    return colors_by_iter

def wl_fingerprint(colors_by_iter):
    fp = []
    for colors in colors_by_iter:
        hist = Counter(colors.values())
        fp.append(tuple(sorted(hist.items())))
    return tuple(fp)

def wl_distinguishes(G, H, T=5, init="uniform"):
    fpG = wl_fingerprint(wl1_colors_nx(G, num_iters=T, init=init))
    fpH = wl_fingerprint(wl1_colors_nx(H, num_iters=T, init=init))
    return fpG != fpH


## From 1-WL to Message Passing: The Expressivity Connection

A typical Message Passing Neural Network (MPNN) update follows the general form:
$$
h_i^{(t+1)} = \phi\Big(h_i^{(t)},\ \mathrm{AGG}\{ \psi(h_i^{(t)}, h_j^{(t)}) : j\in \mathcal{N}(i)\}\Big)
$$

Where:
- **$\text{AGG}$**: A permutation-invariant operator (e.g., $\text{SUM}, \text{MEAN}, \text{MAX}$).
- **$\psi$ and $\phi$**: Learnable functions (typically MLPs).

### The Parallel to 1-WL
The structural similarity between MPNNs and 1-WL is profound:
* **Discrete vs. Continuous:** 1-WL updates **discrete labels** via hashing, while GNNs update **continuous embeddings** via neural layers.
* **Injective Mapping:** If a GNN’s aggregation and update functions are **injective**, the GNN can theoretically reach—but never exceed—the graph-distinguishing power of the 1-WL test.



### Expressivity of Common GNN Architectures

* **GIN (Graph Isomorphism Network):** Specifically designed to be **1-WL equivalent**. By using **sum aggregation** and a MLP (based on the Universal Approximation Theorem), GIN can represent injective functions over multisets.
* **GCN / Mean Aggregators:** Inherently **less expressive** than 1-WL. Mean aggregation can produce identical outputs for different multisets (e.g., a neighborhood of $\{1, 1\}$ vs. $\{1, 1, 1, 1\}$), failing to capture structural cardinality.
* **GAT (Graph Attention Network):** While powerful for feature weighting, GAT is still bounded by 1-WL. It can still suffer from "collisions" where different local structures result in the same aggregated representation.



## Capabilities and Fundamental Limitations of 1-WL

### Strengths of the 1-WL Test
- **Efficiency:** Provides a fast, deterministic refinement of node labels.
- **Local Motif Detection:** Effectively distinguishes graphs with differing local connectivity patterns.
- **Soundness:** If 1-WL distinguishes two graphs, they are **guaranteed** to be non-isomorphic.

### The Failure Case: Regularity and Symmetry
The primary limitation of 1-WL is its inability to distinguish graphs where nodes "see" identical local environments due to high symmetry or regularity.

**Classic Counterexample:**
* **Graph $G$**: A single 6-node cycle ($C_6$).
* **Graph $H$**: Two disjoint 3-node triangles ($C_3 \cup C_3$).



In both cases, every node is **2-regular** (has exactly 2 neighbors). If all nodes start with the same label, 1-WL refinement will never break symmetry; every node in both $G$ and $H$ will receive the exact same sequence of labels. 1-WL will conclude they are "possibly isomorphic," even though their global structures are completely different.

### Consequence for GNNs
Standard MPNNs without **positional encodings** or **unique node IDs** inherit these 1-WL failures. A standard GCN, for instance, would produce identical graph-level embeddings for $C_6$ and $C_3 \cup C_3$.


In the code below, we compare outputs of the 1-WL test and GIN architecture on the same graphs.
- Case A) Graph Isomorphism: same graph $\mathbb{G}$
- Case B) Graph Non-Isomorphism: $\mathbb{G}$ !=  $\mathbb{H}$
- Case C) Study case on failure mode: $\mathbb{C_6}$ (hexagon) != $\mathbb{C_3}$ U $\mathbb{C_3}$ (two triangles)


In [ ]:
def nx_to_pyg(G, x_mode="ones"):
    n = G.number_of_nodes()
    assert set(G.nodes()) == set(range(n)), "Expected node labels 0..n-1"

    edges = list(G.edges())
    edge_index = torch.tensor(edges + [(v, u) for (u, v) in edges], dtype=torch.long).t().contiguous()

    if x_mode == "ones":
        x = torch.ones((n, 1), dtype=torch.float)
    elif x_mode == "degree":
        x = torch.tensor([G.degree[i] for i in range(n)], dtype=torch.float).view(-1, 1)
    else:
        raise ValueError("x_mode must be 'ones' or 'degree'")

    return Data(x=x, edge_index=edge_index)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.data import Data
import networkx as nx
from collections import Counter
import random

# Graph-level wrapper: encoder + pooling ---
class GINGraphModel(nn.Module):
    def __init__(self, in_channels=1, hidden=128, out_channels=128, dropout=0.0, pool="add"):
        super().__init__()
        self.encoder = GINEncoder(in_channels, hidden, out_channels, dropout)
        self.pool = pool

    def forward(self, data: Data):
        z = self.encoder(data.x, data.edge_index)  # [N, out_channels]
        batch = getattr(data, "batch", None)
        if batch is None:
            batch = torch.zeros(z.size(0), dtype=torch.long, device=z.device)

        if self.pool == "add":
            g = global_add_pool(z, batch)
        else:
            raise ValueError("Only pool='add' implemented here.")

        return g.squeeze(0)  # [out_channels] for a single graph

In [ ]:
device = torch.device("cpu")
torch.manual_seed(42)

model = GINGraphModel(in_channels=1, hidden=128, out_channels=128, dropout=0.0, pool="add").to(device)
model.eval()

def gin_embed(G, x_mode="ones"):
    data = nx_to_pyg(G, x_mode=x_mode).to(device)
    with torch.no_grad():
        g = model(data)
    return g

def compare_pair(name, A, B, wl_T=5, wl_init="uniform", x_mode="ones"):
    wl_diff = wl_distinguishes(A, B, T=wl_T, init=wl_init)
    gA = gin_embed(A, x_mode=x_mode)
    gB = gin_embed(B, x_mode=x_mode)
    dist = torch.norm(gA - gB, p=2).item()

    print(f"\n{name}")
    print(f"1-WL distinguishes? {wl_diff} (T={wl_T}, init={wl_init})")
    print(f"GIN graph embedding L2 distance: {dist:.6f} (x_mode={x_mode})")

# Case A: isomorphic pair (C6 vs relabeled C6)
G0 = nx.cycle_graph(6)
perm = list(range(6))
random.seed(0)
random.shuffle(perm)
mapping = {i: perm[i] for i in range(6)}
G0_iso = nx.relabel_nodes(G0, mapping)

# Case B1: non-iso where WL succeeds (C6 vs P6)
G_cycle = nx.cycle_graph(6)
G_path = nx.path_graph(6)

# Case B2: non-iso where WL fails (C6 vs two triangles)
G_two_tri = nx.disjoint_union(nx.cycle_graph(3), nx.cycle_graph(3))

compare_pair("Case A: Isomorphic (C6 vs relabeled C6).\n  ** Expected outcome: 1-WL distinguishes is False (same graph), GIN embedding distance ~ 0.00", G0, G0_iso, wl_T=5, wl_init="uniform", x_mode="ones")
compare_pair("Case B1: Non-isomorphic (C6 vs P6).\n  ** Expected outcome: 1-WL distinguishes is True (graphs are different), GIN embedding distance > 0.00", G_cycle, G_path, wl_T=5, wl_init="uniform", x_mode="ones")
compare_pair("Case B2: Non-isomorphic (C6 vs 2×C3)\n  ** Expected outcome: 1-WL distinguishes is False (failure mode), GIN embedding distance ~ 0.00", G_cycle, G_two_tri, wl_T=5, wl_init="uniform", x_mode="ones")

# Optional: show what happens if you give both WL and GIN degree initialization
compare_pair("Case B2 with degree init/features", G_cycle, G_two_tri, wl_T=5, wl_init="degree", x_mode="degree")


#🥉 Part 3: Equivariance and Invariance

## Testing for Permutation Invariance and Equivariance


- A **GNN <ins>layer**</ins> is **equivariant** to permutations of the set of nodes in the graph; i.e. as we permute the nodes, the node features produced by the GNN must permute accordingly.
- A **GNN <ins>model**</ins> for graph-level property prediction is **invariant** to the permutations of the set of nodes in the graph; i.e. as we permute the nodes, the graph-level property remains unchanged.

A permutation is an **ordering of the nodes** in a graph. In general, there is **no canonical way** of assigning an ordering of the nodes, unlike textual or image data, and we need to ensure that our models are able to principaly handle this **lack of canonical ordering** or permutation of graph nodes.

### Formalism

Let us try to formalise these notions of permutation invariance and equivariance via matrix notation (it is easier that way).

- Let $\mathbf{H} \in \mathbb{R}^{n \times d}$ be a matrix of node features for a given molecular graph, where $n$ is the number of nodes/atoms and each row $h_i$ is the $d$-dimensional feature for node $i$.
- Let $\mathbf{A} \in \mathbb{R}^{n \times n}$ be the adjacency matrix where each entry denotes $a_{ij}$ the presence or absence of an edge between nodes $i$ and $j$.
- Let $\mathbf{F}(\mathbf{H}, \mathbf{A}): \mathbb{R}^{n \times d} \times \mathbb{R}^{n \times n} \rightarrow \mathbb{R}^{n \times d}$ be a **GNN <ins>layer**</ins> that takes as input the node features and adjacency matrix, and returns the **updated node features**.
- Let $f(\mathbf{H}, \mathbf{A}): \mathbb{R}^{n \times d} \times \mathbb{R}^{n \times n} \rightarrow \mathbb{R}$ be a **GNN <ins>model**</ins> that takes as input the node features and adjacency matrix, and returns the **predicted graph-level property**.
- Let $\mathbf{P} \in \mathbb{R}^{n \times n}$ be a **[permutation matrix](https://en.wikipedia.org/wiki/Permutation_matrix)** which has exactly one 1 in every row and column, and 0s elsewhere. Left-multipying $\mathbf{P}$ with a matrix changes the ordering of the rows of the matrix.

### Permuation Equivariance

The GNN <ins>layer</ins> $\mathbf{F}$ is **permuation equivariant** as follows:
$$
\mathbf{F}(\mathbf{PH}, \mathbf{PAP^T}) = \mathbf{P} \ \mathbf{F}(\mathbf{H}, \mathbf{A}).
$$

Another way to formulate the above could be: (1) Consider the updated node features $\mathbf{H'} = \mathbf{F}(\mathbf{H}, \mathbf{A})$. (2) Applying any permutation matrix $\mathbf{P}$ to the input of the GNN layer $\mathbf{F}$ should produce the same result as applying the same permutation on $\mathbf{H'}$:
$$
\mathbf{F}(\mathbf{PH}, \mathbf{PAP^T}) = \mathbf{P} \ \mathbf{H'}
$$

### Permuation Invariance

The GNN <ins>model</ins> $f$ for graph-level prediction is **permutation invariant** as follows:
$$
f(\mathbf{PH}, \mathbf{PAP^T}) = f(\mathbf{H}, \mathbf{A}).
$$

Another way to formulate the above could be: (1) Consider the predicted molecular property $\mathbf{\hat y} = f(\mathbf{H}, \mathbf{A})$. (2) Applying any permutation matrix $\mathbf{P}$ to the input of the GNN model $f$ should produce the same result as not applying it:
$$
f(\mathbf{PH}, \mathbf{PAP^T}) = \mathbf{\hat y}.
$$



In [ ]:
# Toy graph generator
def make_toy_graph(num_nodes=12, feat_dim=16):
    # Ring + a few chords, made undirected by adding both directions
    edges = []
    for i in range(num_nodes):
        j = (i + 1) % num_nodes
        edges.append((i, j))
        edges.append((j, i))
    for i in range(0, num_nodes, 3):
        j = (i + 4) % num_nodes
        edges.append((i, j))
        edges.append((j, i))

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    x = torch.randn(num_nodes, feat_dim)
    return Data(x=x, edge_index=edge_index)


# Permutation helpers
def consistent_permute(data: Data, perm: torch.Tensor):
    """
    perm: new_index -> old_index
    Returns:
      data_perm: permuted graph
      invperm: old_index -> new_index
    """
    N = data.num_nodes
    invperm = torch.empty_like(perm)
    invperm[perm] = torch.arange(N, device=perm.device)

    x_perm = data.x[perm]
    edge_index_perm = invperm[data.edge_index]  # relabel endpoints
    return Data(x=x_perm, edge_index=edge_index_perm), invperm

def inconsistent_permute(data: Data, perm: torch.Tensor):
    """Shuffles node features but does NOT relabel edges (breaks correctness)."""
    x_perm = data.x[perm]
    return Data(x=x_perm, edge_index=data.edge_index)


# Metrics: equivariance/invariance errors
@torch.no_grad()
def compute_reference(model, data):
    z = model.encode(data.x, data.edge_index)
    batch = torch.zeros(data.num_nodes, dtype=torch.long, device=data.x.device)
    g = global_mean_pool(z, batch).squeeze(0)

    # fixed edge queries for edge-level equivariance
    edge_label_index = torch.tensor([[0, 1, 2, 3, 4],
                                     [5, 6, 7, 8, 9]], dtype=torch.long, device=data.x.device)
    logits = model.decode(z, edge_label_index)
    return z, g, edge_label_index, logits

@torch.no_grad()
def measure_errors(model, data_ref, z_ref, g_ref, edge_label_index_ref, logits_ref, data_perm, invperm=None):
    """
    Returns (node_equiv_error, edge_equiv_error, graph_invar_error).
    For consistent perms, provide invperm; for inconsistent, leave invperm=None.
    """
    z_p = model.encode(data_perm.x, data_perm.edge_index)

    # Graph invariance error
    batch = torch.zeros(data_perm.num_nodes, dtype=torch.long, device=data_perm.x.device)
    g_p = global_mean_pool(z_p, batch).squeeze(0)
    graph_err = (g_p - g_ref).abs().max().item()

    # Node equivariance + edge equivariance only make sense with consistent permutation
    if invperm is not None:
        # Recover perm from invperm (perm is inverse mapping)
        perm = torch.empty_like(invperm)
        perm[invperm] = torch.arange(invperm.numel(), device=invperm.device)

        node_err = (z_p - z_ref[perm]).abs().max().item()

        edge_label_index_p = invperm[edge_label_index_ref]
        logits_p = model.decode(z_p, edge_label_index_p)
        edge_err = (logits_p - logits_ref).abs().max().item()
    else:
        node_err = float("nan")
        edge_err = float("nan")

    return node_err, edge_err, graph_err

def summarize(vals):
    vals = [v for v in vals if not (isinstance(v, float) and (v != v))]  # drop NaNs
    if not vals:
        return None
    t = torch.tensor(vals)
    return float(t.min()), float(t.median()), float(t.max())


# Demo
feat_dim = 16
model = LinkPredictor(in_channels=feat_dim, hidden=64, z_dim=64, dropout=0.2).to(device)

model.eval()

data = make_toy_graph(num_nodes=12, feat_dim=feat_dim).to(device)

z_ref, g_ref, edge_label_index_ref, logits_ref = compute_reference(model, data)

print("\nSingle-shuffle sanity check (consistent permutation):")
perm = torch.randperm(data.num_nodes, device=device)
data_p, invperm = consistent_permute(data, perm)
node_err, edge_err, graph_err = measure_errors(
    model, data, z_ref, g_ref, edge_label_index_ref, logits_ref, data_p, invperm=invperm
)
print("Node equivariance error (max abs):", node_err)
print("Edge equivariance error (max abs):", edge_err)
print("Graph invariance error (max abs):", graph_err)

print("\nSingle-shuffle sanity check (inconsistent permutation):")
perm = torch.randperm(data.num_nodes, device=device)
data_bad = inconsistent_permute(data, perm).to(device)
node_err_bad, edge_err_bad, graph_err_bad = measure_errors(
    model, data, z_ref, g_ref, edge_label_index_ref, logits_ref, data_bad, invperm=None
)
print("Graph invariance error (max abs) [should be > 0]:", graph_err_bad)


# Stress test over many shuffles
def run_trials(num_trials=50, consistent=True):
    node_errs, edge_errs, graph_errs = [], [], []
    for _ in range(num_trials):
        perm = torch.randperm(data.num_nodes, device=device)
        if consistent:
            data_p, invperm = consistent_permute(data, perm)
            ne, ee, ge = measure_errors(
                model, data, z_ref, g_ref, edge_label_index_ref, logits_ref, data_p, invperm=invperm
            )
        else:
            data_p = inconsistent_permute(data, perm).to(device)
            ne, ee, ge = measure_errors(
                model, data, z_ref, g_ref, edge_label_index_ref, logits_ref, data_p, invperm=None
            )

        node_errs.append(ne)
        edge_errs.append(ee)
        graph_errs.append(ge)

    return summarize(node_errs), summarize(edge_errs), summarize(graph_errs)

print("\nStress test: CONSISTENT shuffling (should be ~0 within tolerance)")
ne_s, ee_s, ge_s = run_trials(num_trials=100, consistent=True)
print("Node equivariance error  (min/med/max):", ne_s)
print("Edge equivariance error  (min/med/max):", ee_s)
print("Graph invariance error   (min/med/max):", ge_s)

print("\nStress test: INCONSISTENT shuffling (graph invariance should break)")
ne_b, ee_b, ge_b = run_trials(num_trials=100, consistent=False)
print("Graph invariance error   (min/med/max):", ge_b)

# Conclusions

Today, we have learnt how to code a Message Passing GNN (GIN) and applied it to the task of link prediction (predicting whether two nodes are connected). Then, we dived into expressivity (a model's ability to learn and represent complex functions over graph-structured data) and tested it with WL-1. Finally, we explore robustness to input variations of GNNs (an important property, given that graphs are not-ordered) looking into both layer-equivariance (shuffled input at layer level reflect in the layer output accordingly) and model-invariance (shuffled input at model level should not impact predictions).